In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import prepare_dataframe, generate_training_dataframe, split_dataframe, manage_prediction, create_real_pred_df, plot_tds, probability_within_radius

from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures

from scipy.stats import norm

# North Pacific Ocean Tropical Depressions Novel Forecast Model

## Abstract

Building upon the foundational [analysis of the effects of the El Niño-Southern Oscillation (ENSO) on tropical depressions (TDs) in the North Pacific Ocean](https://github.com/StanDobrev11/enso_effect_on_npacific_tds), this current project takes the research a step further by developing a machine learning (ML) forecast model. The aim is to predict the track, intensity, and progression of tropical depressions based on their initial characteristics and ENSO phases.

In the previous project, correlations between ENSO phases and various tropical depression characteristics, such as frequency, intensity, and storm tracks were successfully identified, especially in the NE Pacific region. It was established that El Niño and La Niña events have discernible impacts on where and how frequently TDs form, as well as their overall intensity across the North Pacific region. This prior work provided valuable insights into how different ENSO conditions influence tropical depressions, laying a strong foundation for more advanced predictive modeling.

## Introduction

### Brief explanation of ENSO and TDs

The **El Niño-Southern Oscillation (ENSO)** is a climate phenomenon characterized by fluctuations in sea surface temperatures and atmospheric conditions in the Pacific Ocean. ENSO consists of three phases: **El Niño**, when ocean waters in the central and eastern Pacific are warmer than average; **La Niña**, when these waters are cooler than normal; and a **Neutral** phase when conditions are closer to the long-term average. ENSO has significant global impacts, affecting weather patterns, rainfall, and storm activity.

A **Tropical Depression (TD)** is a low-pressure weather system that typically develops around 5 degrees north or south of the equator. These systems are the early stage of tropical cyclones and can intensify into stronger storms, such as tropical storms or hurricanes, depending on favorable atmospheric and oceanic conditions. Key factors influencing the development of TDs include sea surface temperature, wind shear, latent heat, moisture, ENSO phase and the Coriolis effect, which helps initiate the system's rotation.

The current project focuses on the development of a machine learning-based forecast model designed to predict the future position, wind speed, and pressure of newly formed tropical depressions at 6-hour intervals. By leveraging historical tropical depression data and known ENSO phases, the model aims to predict the following:

- **TD Track**: The latitudinal and longitudinal coordinates of the storm’s future location.
- **Wind Speed**: Changes in the storm’s wind intensity.
- **Minimum Central Pressure**: Evolution of the storm’s pressure, which is critical for determining its potential strength.
- **Velocity and Direction**: Predict the movement speed and direction of the tropical depression.

Furthermore, as the tropical depression develops, newly acquired data will be incorporated into the model to improve the accuracy of forecasting its further evolution.

### Goals

1. The project aims to develope a series of machine learning models that will predict the future location, wind speed, pressure, and velocity of a tropical depression for the next 6, 12, 18, and 24 hours.
2. Compare the accuracy of different ML models to determine which performs best across varying prediction intervals, with the expectation that the 6-hour prediction will be the most reliable and the 24-hour forecast less so.
3. Evaluate the performance of the models in terms of bias, variance, and overall error using metrics like Mean Squared Error (MSE) and R-squared scores.
4. Optimize the models through grid search and other tuning methods to ensure the best possible performance.

### Steps to be Followed

1. **Data Preparation and Feature Engineering**: The datasets from the previous project will be used. Some of the features will be transformed and scaled, others will be engineered.

2. **Model Selection, Training and Testing**: The machine learning algorithms to be evaluated are Random Forest Regressor and Gradient Boosting Models (XGBoost). The dataset will be split into training and test sets, with cross-validation to ensure model robustness. Separate models will be trained for 6, 12, 18, and 24-hour prediction intervals, with the recursive use of previous predictions for longer intervals.

4. **Model Evaluation**: The performance of the models will be evaluated using metrics such as MSE, R-squared, and bias-variance tradeoffs. The accuracy of the predictions will be analyzed, particularly regarding the 6-hour and 24-hour predictions, with an expectation of decreasing accuracy for longer intervals.

5. **Optimization**: Hyperparameter tuning will be performed using grid search or randomized search to improve the models’ performance, especially in reducing prediction errors.

## Data Preparation and Feature Engineering

### Data Sources

The data used in this project comes from multiple reputable sources, ensuring comprehensive coverage of both tropical depression (TD) activity and environmental factors like sea surface temperature (SST) anomalies:

The SST anomaly data was obtained from the NASA Earth Data AQUA MODIS satellite, providing high-resolution sea surface temperature measurements. These anomalies reflect deviations from normal sea temperatures, which are critical for understanding the effects of El Niño and La Niña events. The data is in .nc format

Data for tropical depressions in the Northwest Pacific was sourced from the Japan Meteorological Agency (JMA). This dataset includes information on each tropical depression's intensity, minimum pressure, wind speeds, latitude, and longitude.

The National Hurricane Center (NHC) provided tropical depression data for the Northeast and Central Pacific, which is similar to the JMA data in terms of recorded metrics like wind speed, pressure, and storm tracks.

The ENSO phases (El Niño, La Niña, and Neutral) were derived from the ONI table provided by the Climate Prediction Center (CPC). The ONI is calculated as the rolling three-month mean SST anomaly in the Niño 3.4 region and is commonly used to identify ENSO phases.

The GSHHS (Global Self-consistent, Hierarchical, High-resolution Shoreline Database) was also utilized in this project to provide accurate geographical boundaries and coastlines for visualizing tropical depression tracks. This dataset offers detailed representations of global shorelines, which are essential for plotting storm trajectories in relation to landmasses and understanding potential landfall locations. GSHHS data ensures that the geospatial analysis of tropical depressions remains accurate and visually consistent when overlaying storm tracks on global maps.

The dataset from the previous project, which includes key information such as TD latitude, longitude, wind speed, minimum central pressure, and ENSO phase, will be further refined. Data cleaning steps included handling missing values, standardizing units, and aligning time formats. The full details of the data cleaning process can be found in the GitHub repository [here](https://github.com/StanDobrev11/enso_effect_on_npacific_tds), where the code for preprocessing is documented and accessible. This ensures the dataset is ready for further analysis and machine learning modeling.

Remaining missing wind data for the JMA dataset will be generated using a Random Forest model, and similarly, missing pressure data for the NHC dataset will be completed using the same method. Additional derived features, such as velocity and direction, will be included, with necessary transformations like trigonometric calculations for geographical coordinates. Features such as latitude, longitude, ENSO phase (one-hot encoded), wind speed, pressure, and the derived velocity and direction will be used to train the models, ensuring a comprehensive representation of tropical depression dynamics.

### Data Preparation

For data preparation, the function **prepare_dataframe()** is imported. Clear explanation of the transformation, performed on the dataframe, are described on the function's docstring. First, we will fill out the missing **wind** values from the JMA dataset.

In [3]:
# read jma data
jma = pd.read_csv('data/csv_ready/jma_td.csv', index_col=0)

# converting the longitude to standart values [-180:180]
jma['lon'] = jma['lon'].apply(lambda x: 360 - x if x > 180 else x)

# drop first row because first and second are same
jma = jma.drop(jma.index[0])

# add values for the wind, assuming minimum wind of the TD to be 35kn
jma.loc[jma['min_pressure_mBar'] >= 980, 'max_wind_kn'] = 35

In [4]:
jma = prepare_dataframe(jma)

In this project, Cartesian coordinates are used to transform the geographical latitude and longitude into a format that is easier for machine learning models to process. Latitude and longitude represent curved surfaces on Earth, but Cartesian coordinates allow the model to work with a flat, linear representation, making it easier to capture spatial relationships in a way that ML algorithms can interpret effectively. This helps improve the accuracy of the predictions.

Transforming the direction into sine and cosine components is done to avoid discontinuities that arise from using angles directly (since 0° and 360° are mathematically the same direction but numerically far apart). By using sine and cosine, the model can treat direction as a smooth, continuous feature, which better represents the cyclical nature of angles and helps improve learning efficiency.

Plane Sailing is a simplified navigational technique used to estimate the course and distance between two points on Earth, assuming that the Earth is flat for small distances. This method works by treating the meridians (lines of longitude) and parallels (lines of latitude) as straight lines, which allows for easy calculations using basic trigonometry. Plane sailing assumes a flat Earth, which introduces only small errors when dealing with short distances. Over small distances, the curvature of the Earth is minimal and can be safely ignored. This makes plane sailing a convenient and fast method for navigation over distances typically less than 600 nautical miles (NM) and therefore, for calculating course and speed of TDs for time intervals of 6 hours.

In the case of tropical depressions (TD), the speed and direction (or velocity and bearing) are relevant to describe the movement from one point to the next. When at point 0 (initial observation), we don't know the speed and direction until next observation position after 6 hours. Shifting the speed and direction by -1 ensures that at each point, the actual speed and direction required to reach the next point are used and the very last observation will have the speed and direction set to 0 to reflect TD's disipation.

In [ ]:
jma[:10]

In [ ]:
# the result of this line shows how many observation are missing wind value
jma.max_wind_kn[jma.max_wind_kn == 0].count()

In [ ]:
# separate the dataframe
available_data_jma = jma[jma['max_wind_kn'] > 0]
predicted_data_jma = jma[jma['max_wind_kn'] == 0]

X_raw_jma = available_data_jma[['min_pressure_mBar', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]
y_raw_jma = available_data_jma['max_wind_kn']

# splitting the data 80/20 sounds reasonable. There are abt 62000 valid observations
X_train_jma, X_test_jma, y_train_jma, y_test_jma = train_test_split(X_raw_jma, y_raw_jma, test_size=0.2, random_state=42)
X_pred_jma = predicted_data_jma[['min_pressure_mBar', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]

In [ ]:
# building the pipeline
preprocessor_jma = ColumnTransformer([
    ('enso', OneHotEncoder(), ['enso']),
    ('scaler', MinMaxScaler(), ['min_pressure_mBar', 'velocity_kn']),
], remainder='passthrough', force_int_remainder_cols=False)

pipeline_jma = Pipeline([
    ('preprocess', preprocessor_jma),  # Apply scaling and encoding
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=97))
])

Splitting the data 80/20 means using 80% of the dataset (about 49,600 observations) for training the model and the remaining 20% (about 12,400 observations) for testing. This is a common practice to ensure the model has enough data to learn from while still reserving a significant portion for evaluating its performance on unseen data. The train_test_split function with random_state=42 ensures that the split is reproducible, meaning the same split can be obtained each time the code is run.

This pipeline automates the process of preparing the data and training the model for tropical depression forecasting.

First, the preprocessor step handles the data transformations: **ENSO phases** (El Niño, La Niña, Neutral) are one-hot encoded to convert the categorical data into a format that the model can understand.
**MinPressure** and **Velocity** are scaled using **Min-Max scaling** to normalize the values, making them easier for the model to interpret.
Next, the **RandomForestRegressor** is applied to the preprocessed data. This model uses multiple decision trees to predict the next values for wind speed, pressure, and coordinates based on the historical data and ENSO phase. The pipeline combines all of these steps, making it easier to train and test the model efficiently.

Then we fit the training data to the pipeline and evaluate the model:

In [ ]:
pipeline_jma.fit(X_train_jma, y_train_jma)

In [ ]:
pipeline_jma.score(X_test_jma, y_test_jma)

In [ ]:
y_pred = pipeline_jma.predict(X_test_jma)
print(f'Mean Squared Error: {mean_squared_error(y_test_jma, y_pred)}')

The result shows that the RandomForestRegressor model is performing very well on the test data, with a high accuracy score of 0.99 (which indicates the model is correctly predicting 99% of the variance in the data). The Mean Squared Error (MSE) of 3.26 also indicates that, on average, the model's predictions deviate by about 3.26 units (kn) from the actual values. This is a low error value, further suggesting that the model is making precise predictions. The value of 3.26 kn deviation in the range of 35 - 140 kn of wind is acceptable.

Further evaluation of the model's reliability and potential overfitting will not be conducted, as the model's output is intended solely for a one-time use to fill missing data in the dataset.

With that in mind, missing wind data will be now applied to the jma dataset and will check if there are any '0' values remaining.

In [ ]:
predicted_winds = pipeline_jma.predict(X_pred_jma)
jma.loc[jma['max_wind_kn'] == 0, 'max_wind_kn'] = predicted_winds.astype(int)

In [ ]:
jma.max_wind_kn[jma.max_wind_kn == 0].count()

In [ ]:
importances = pipeline_jma.named_steps['regressor'].feature_importances_
feature_names = pipeline_jma.named_steps['preprocess'].get_feature_names_out()
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

The feature importance table shows how much each feature contributes to the model's predictive power. In this case, min_pressure_mBar is the most important feature by far, with a feature importance score of 0.986759. This suggests that the model relies heavily on the minimum central pressure to make accurate predictions. The remaining features, including the x, y, z cartesian coordinates, velocity_kn, direction_sin, direction_cos, and the ENSO phase encoded as one-hot variables, have much lower importance, indicating that their contribution to the model's performance is minimal in comparison. This suggests that the pressure plays a critical role in determining the outcome, while the other features provide only marginal improvements.

A similar operation will be performed on the NHC dataset; however, in this case, the missing values are for minimum central pressure. The pressure values that are to be modeled, have a negative value.

In [ ]:
nhc = pd.read_csv('data/csv_ready/ne_pacific_td.csv', index_col=0)
# converting the longitude to standart values [-180:180]
nhc['lon'] = 180 - nhc['lon']

In [ ]:
nhc = prepare_dataframe(nhc)

In [ ]:
# building the pipeline
preprocessor_nhc = ColumnTransformer([
    ('enso', OneHotEncoder(), ['enso']),
    ('scaler', MinMaxScaler(), ['max_wind_kn', 'velocity_kn']),
], remainder='passthrough', force_int_remainder_cols=False)

pipeline_nhc = Pipeline([
    ('preprocess', preprocessor_nhc),  # Apply scaling and encoding
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=97))
])

In [ ]:
# separate the dataframe
available_data_nhc = nhc[nhc['min_pressure_mBar'] > 0]
predicted_data_nhc = nhc[nhc['min_pressure_mBar'] < 0]

X_raw_nhc = available_data_nhc[['max_wind_kn', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]
y_raw_nhc = available_data_nhc['min_pressure_mBar']

# splitting the data 80/20 sounds reasonable. There are abt 62000 valid observations
X_train_nhc, X_test_nhc, y_train_nhc, y_test_nhc = train_test_split(X_raw_nhc, y_raw_nhc, test_size=0.2, random_state=97)
X_pred_nhc = predicted_data_nhc[['max_wind_kn', 'velocity_kn', 'direction_sin', 'direction_cos', 'x', 'y', 'z', 'enso']]

In [ ]:
pipeline_nhc.fit(X_train_nhc, y_train_nhc)

In [ ]:
pipeline_nhc.score(X_test_nhc, y_test_nhc)

In [ ]:
y_pred = pipeline_nhc.predict(X_test_nhc)
print(f'Mean Squared Error: {mean_squared_error(y_test_nhc, y_pred)}')

In [ ]:
predicted_pressure = pipeline_nhc.predict(X_pred_nhc)
nhc.loc[nhc['min_pressure_mBar'] < 0, 'min_pressure_mBar'] = predicted_pressure.astype(int)

In [ ]:
nhc.min_pressure_mBar[nhc.min_pressure_mBar < 0].count()

In [ ]:
importances = pipeline_nhc.named_steps['regressor'].feature_importances_
feature_names = pipeline_nhc.named_steps['preprocess'].get_feature_names_out()
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
print(feature_importance_df.sort_values(by='Importance', ascending=False))

Now that the missing values for both datasets, JMA and NHC are completed, they are saved for further usage in ML.

In [ ]:
# dropping columns not required and saving the datasets
jma = jma.drop(columns=['direction_sin', 'direction_cos', 'x', 'y', 'z'])
nhc = nhc.drop(columns=['direction_sin', 'direction_cos', 'x', 'y', 'z'])
jma.to_csv('data/csv_ready/jma_training')
nhc.to_csv('data/csv_ready/nhc_training')

## Model Selection, Training & Testing

### Conceptual Clarification of Prediction Setup
The idea is to predict the next position, speed, and course based on how the storm reached the current position. For this:

**Training Data Setup:**

- Each row should include information on the storm's movement (speed and course) toward the current position as lagged features.
- The target variables (latitude, longitude, wind speed and minimum pressure) represent the next position's attributes.
Prediction:
- Input the current position and movement characteristics (speed, course, and lagged features) to predict the attributes of the next position.

The add_lags function creates lagged features by shifting data for specific columns grouped by storms (e.g., grouped by a unique identifier like storm_id). However, to make it more precise, the function is deriving speed and course directly from latitude/longitude.

In [ ]:
def add_lags(df, features, n_lags, group_col):
    """
    Adds lagged features for time series grouped by a specific column.

    Args:
    - df (DataFrame): The input DataFrame.
    - features (list): List of feature names to lag.
    - n_lags (int): Number of lag periods.
    - group_col (str): Column to group by (e.g., 'storm_id').

    Returns:
    - DataFrame: The DataFrame with lagged features added.
    """
    for lag in range(1, n_lags + 1):
        for feature in features:
            df[f"{feature}_lag_{lag}"] = df.groupby(group_col)[feature].shift(lag)
    # Drop rows where lagged values are NaN due to shifting
    return df.dropna(subset=[f"{feature}_lag_{lag}" for feature in features])

### XGBRegressor Workflow and Adaptation for Time Series Prediction

XGBRegressor is incorporated into the project for its ability to model complex, non-linear relationships efficiently while maintaining robustness and scalability. Unlike linear models like SGDRegressor, XGBRegressor excels at handling structured time-series data by leveraging gradient boosting techniques to optimize predictions iteratively. This makes it a strong candidate for modeling tropical depression (TD) tracks and intensities.

For this project, XGBRegressor provides the flexibility to handle mixed data types (e.g., categorical and continuous features like ENSO phases, wind speed, latitude, and longitude) and allows for advanced feature engineering and custom loss functions. Its tree-based nature eliminates the need for extensive feature scaling, simplifying the preprocessing pipeline.

#### Workflow


In [5]:
df = pd.read_csv('data/csv_ready/jma_training', index_col=0)

In [6]:
prep_data = FunctionTransformer(func=prepare_dataframe)
prep_train_data = FunctionTransformer(func=generate_training_dataframe)

In [7]:
preprocessor = ColumnTransformer([
        ('enso', OneHotEncoder(), ['enso']),  # One-hot encode the ENSO feature
        ('poly', PolynomialFeatures(degree=3, include_bias=False), slice(0, None)),
    ], remainder='passthrough', force_int_remainder_cols=False)
scaler = MinMaxScaler()
regressor = MultiOutputRegressor(SGDRegressor(learning_rate='adaptive', max_iter=3000, penalty='l1', alpha=1e-6, tol=1e-5, random_state=97))

In [8]:
jma_pred_pipeline_sgd = Pipeline([
        ('prep_data', prep_data),
        ('preprocessor', preprocessor),
        ('scaler', scaler),
        ('regressor', regressor)
    ])

jma_train_pipeline_sgd = Pipeline([
        ('preprocessor', preprocessor),
        ('scaler', scaler),
        ('regressor', regressor)
    ])

In [10]:
# df_trn = prep_data.transform(df)

In [11]:
# df_trn = prep_train_data.transform(df_trn)
df_trn = pd.read_csv('data/csv_ready/df_trn.csv')
X_train, X_test, y_train, y_test = next(split_dataframe(df_trn, splitter='gss'))

In [12]:
jma_train_pipeline_sgd.fit(X_train, y_train)

In [13]:
jma_train_pipeline_sgd.score(X_test, y_test)

In [14]:
X_test.head()

In [15]:
jma = pd.read_csv('data/csv_ready/jma_training', index_col=0)

In [17]:
plot_tds(jma, X_test, model=jma_pred_pipeline_sgd, n_samples=3, random_state=97)

In [18]:
y_test

In [19]:
X_pred = jma[jma.index == '2023-10-14 18:00:00']

In [20]:
X_expected = jma[jma.index == '2023-10-15 00:00:00']

In [21]:
X_pred

In [22]:
X_expected

In [23]:
jma_pred_pipeline_sgd.predict(X_pred)

In [24]:
manage_prediction(X_pred, jma_pred_pipeline_sgd)

In [ ]:
df_ = create_real_pred_df(jma, X_test, model=jma_pred_pipeline_sgd, n_samples=3, random_state=97)

**Method to Calculate Probability of Predicted Position**

To calculate the probability of the predicted position being within a certain range of the actual position, we can use a statistical approach based on error distributions. The error distribution is calculated between predicted and real positions for the test dataset. Then we fit a normal distribution to the error values in terms of latitude and longitude differences. The standard deviation (σ) of the error distribution is used to estimate the confidence interval.

Using the fitted distribution, we calculate the probability that the actual position falls within a certain radius of the predicted position.



In [ ]:
df_= probability_within_radius(df_, 10)

In [ ]:
df_.probability_within_radius.min(), df_.probability_within_radius.max()

In [ ]:
df_[df_.group == 127]

In [ ]:
X_test

In [ ]:
X_test[X_test.velocity_kn > 15][30:50]

In [ ]:
X_pred

In [ ]:
X_pred = jma[jma.index == '1952-10-11 12:00:00']
X_expected = jma[jma.index == '1952-10-11 18:00:00']
manage_prediction(X_pred, jma_pred_pipeline_sgd)

In [ ]:
X_expected